In [2]:
import pandas as pd
import numpy as np
import duckdb

In [8]:
df1 = pd.read_csv('./data/exam1.csv', encoding = 'utf-8')
df2 = pd.read_csv('./data/exam2.csv', encoding = 'utf-8')

In [9]:
df1

,empno,ename,dname,score1,score2
0,1001,김길동,c01,100.0,70.0
1,1002,나길동,c01,90.0,80.0
2,1003,다길동,c02,80.0,90.0
3,1004,라길동,c02,NaN,60.0
4,1005,마길동,c02,90.0,90.0
5,1006,바길동,c01,70.0,90.0
6,1007,상길동,c03,80.0,80.0
7,1008,이길동,c01,90.0,NaN
8,1009,장길동,c01,60.0,90.0
9,1010,차길동,c03,90.0,70.0


In [10]:
df2

,deptno,dname
0,c01,컴퓨터
1,c02,미디어
2,c03,보안


In [11]:
df2.loc[3,["deptno", "dname"]] = ['c04', '정보통신']

In [12]:
df2

,deptno,dname
0,c01,컴퓨터
1,c02,미디어
2,c03,보안
3,c04,정보통신


In [13]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   empno   14 non-null     int64  
 1   ename   14 non-null     object 
 2   dname   14 non-null     object 
 3   score1  13 non-null     float64
 4   score2  12 non-null     float64
dtypes: float64(2), int64(1), object(2)
memory usage: 688.0+ bytes


In [16]:
df1.isna().sum()

empno     0
ename     0
dname     0
score1    1
score2    2
dtype: int64

In [19]:
df1.fillna({'score1':0,'score2':0}, inplace=True)

In [20]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   empno   14 non-null     int64  
 1   ename   14 non-null     object 
 2   dname   14 non-null     object 
 3   score1  14 non-null     float64
 4   score2  14 non-null     float64
dtypes: float64(2), int64(1), object(2)
memory usage: 688.0+ bytes


In [21]:
df1.isna().sum()

empno     0
ename     0
dname     0
score1    0
score2    0
dtype: int64

In [28]:
df1.groupby('dname').agg({'score1':'mean',
                          'score2':'mean'}).round(2)

,score1,score2
dname,,
c01,81.67,68.33
c02,56.67,80.00
c03,78.00,54.00


In [32]:
df1.groupby('dname').agg(score1_mean = ('score1','mean'),
                         score2_mean = ('score2','mean')).round(2)

,score1_mean,score2_mean
dname,,
c01,81.67,68.33
c02,56.67,80.00
c03,78.00,54.00


In [36]:
df1.groupby('dname').agg({'score1':['min', 'max'],
                          'score2':'mean'}).round(2)

score1        score2
         min    max   mean
dname                     
c01     60.0  100.0  68.33
c02      0.0   90.0  80.00
c03     70.0   90.0  54.00

In [33]:
df1.groupby('dname').agg(score1_min = ('score1','min'),
                         score1_max = ('score1','max'),
                         score2_mean = ('score2','mean')).round(2)

,score1_min,score1_max,score2_mean
dname,,,
c01,60.0,100.0,68.33
c02,0.0,90.0,80.00
c03,70.0,90.0,54.00


In [38]:
duckdb.query('''
    select dname, min(score1), max(score1), avg(score2) 
        from df1
        group by dname

''').df()

,dname,min(score1),max(score1),avg(score2)
0,c01,60.0,100.0,68.333333
1,c02,0.0,90.0,80.000000
2,c03,70.0,90.0,54.000000


In [49]:
df2

,deptno,dname
0,c01,컴퓨터
1,c02,미디어
2,c03,보안
3,c04,정보통신


In [56]:
pd.merge(df1, df2, how = 'left', left_on='dname', right_on='deptno')

,empno,ename,dname_x,score1,score2,deptno,dname_y
0,1001,김길동,c01,100.0,70.0,c01,컴퓨터
1,1002,나길동,c01,90.0,80.0,c01,컴퓨터
2,1003,다길동,c02,80.0,90.0,c02,미디어
3,1004,라길동,c02,0.0,60.0,c02,미디어
4,1005,마길동,c02,90.0,90.0,c02,미디어
5,1006,바길동,c01,70.0,90.0,c01,컴퓨터
6,1007,상길동,c03,80.0,80.0,c03,보안
7,1008,이길동,c01,90.0,0.0,c01,컴퓨터
8,1009,장길동,c01,60.0,90.0,c01,컴퓨터
9,1010,차길동,c03,90.0,70.0,c03,보안


In [58]:
pd.merge(df1, df2, how = 'right', left_on='dname', right_on='deptno')

,empno,ename,dname_x,score1,score2,deptno,dname_y
0,1001.0,김길동,c01,100.0,70.0,c01,컴퓨터
1,1002.0,나길동,c01,90.0,80.0,c01,컴퓨터
2,1006.0,바길동,c01,70.0,90.0,c01,컴퓨터
3,1008.0,이길동,c01,90.0,0.0,c01,컴퓨터
4,1009.0,장길동,c01,60.0,90.0,c01,컴퓨터
5,1012.0,타길동,c01,80.0,80.0,c01,컴퓨터
6,1003.0,다길동,c02,80.0,90.0,c02,미디어
7,1004.0,라길동,c02,0.0,60.0,c02,미디어
8,1005.0,마길동,c02,90.0,90.0,c02,미디어
9,1007.0,상길동,c03,80.0,80.0,c03,보안


In [57]:
duckdb.query('''
    select df1.* , df2.dname as dname2
        from df1
        left join df2 on df1.dname = df2.deptno
''').df()

,empno,ename,dname,score1,score2,dname2
0,1001,김길동,c01,100.0,70.0,컴퓨터
1,1002,나길동,c01,90.0,80.0,컴퓨터
2,1003,다길동,c02,80.0,90.0,미디어
3,1004,라길동,c02,0.0,60.0,미디어
4,1005,마길동,c02,90.0,90.0,미디어
5,1006,바길동,c01,70.0,90.0,컴퓨터
6,1007,상길동,c03,80.0,80.0,보안
7,1008,이길동,c01,90.0,0.0,컴퓨터
8,1009,장길동,c01,60.0,90.0,컴퓨터
9,1010,차길동,c03,90.0,70.0,보안


In [59]:
duckdb.query('''
    select df1.* , df2.dname as dname2
        from df1
        right join df2 on df1.dname = df2.deptno
''').df()

,empno,ename,dname,score1,score2,dname2
0,1001,김길동,c01,100.0,70.0,컴퓨터
1,1002,나길동,c01,90.0,80.0,컴퓨터
2,1003,다길동,c02,80.0,90.0,미디어
3,1004,라길동,c02,0.0,60.0,미디어
4,1005,마길동,c02,90.0,90.0,미디어
5,1006,바길동,c01,70.0,90.0,컴퓨터
6,1007,상길동,c03,80.0,80.0,보안
7,1008,이길동,c01,90.0,0.0,컴퓨터
8,1009,장길동,c01,60.0,90.0,컴퓨터
9,1010,차길동,c03,90.0,70.0,보안


In [61]:
df1.merge(df2, how = 'right', left_on='dname', right_on='deptno')

,empno,ename,dname_x,score1,score2,deptno,dname_y
0,1001.0,김길동,c01,100.0,70.0,c01,컴퓨터
1,1002.0,나길동,c01,90.0,80.0,c01,컴퓨터
2,1006.0,바길동,c01,70.0,90.0,c01,컴퓨터
3,1008.0,이길동,c01,90.0,0.0,c01,컴퓨터
4,1009.0,장길동,c01,60.0,90.0,c01,컴퓨터
5,1012.0,타길동,c01,80.0,80.0,c01,컴퓨터
6,1003.0,다길동,c02,80.0,90.0,c02,미디어
7,1004.0,라길동,c02,0.0,60.0,c02,미디어
8,1005.0,마길동,c02,90.0,90.0,c02,미디어
9,1007.0,상길동,c03,80.0,80.0,c03,보안


In [63]:
df1.merge(df2, how = 'left', left_on='dname', right_on='deptno', suffixes = ('deptno', 'dname'))

,empno,ename,dnamedeptno,score1,score2,deptno,dnamedname
0,1001,김길동,c01,100.0,70.0,c01,컴퓨터
1,1002,나길동,c01,90.0,80.0,c01,컴퓨터
2,1003,다길동,c02,80.0,90.0,c02,미디어
3,1004,라길동,c02,0.0,60.0,c02,미디어
4,1005,마길동,c02,90.0,90.0,c02,미디어
5,1006,바길동,c01,70.0,90.0,c01,컴퓨터
6,1007,상길동,c03,80.0,80.0,c03,보안
7,1008,이길동,c01,90.0,0.0,c01,컴퓨터
8,1009,장길동,c01,60.0,90.0,c01,컴퓨터
9,1010,차길동,c03,90.0,70.0,c03,보안


In [54]:
df2

,deptno,dname
0,c01,컴퓨터
1,c02,미디어
2,c03,보안
3,c04,정보통신
